In [97]:
## FILE MANAGEMENT
import os

## DATETIME
import datetime as dt
from datetime import date

## DATA TABLES
import math
import numpy as np
import pandas as pd

## Utility function for sorting numeric text without leading zeroes.
import re
def natural_sort_key(text):
    return [int(part) if part.isdigit() else part.lower() for part in re.split(r'(\d+)', text)]

# 💾 Data

Download all 12 of the .xls tables for the 11 regions and the statewide projects from the NYS Department of Transportation into a new folder.

**STIP Project List and Data Download:**
https://www.dot.ny.gov/programs/stip/stip-project-rpt

#### Load and Concatenate the STIP Tables

##### Function: Load STIP tables as a list

In [98]:
def load_stip_tables(input_folder, update_date):
  ## Initial Sort of Files in Folder
  input_list = os.listdir(input_folder)
  input_list.sort()
  print("Initial Sort of Files:\t", input_list)

  ## Natural Sort of Files in Folder
  input_list.sort(key=natural_sort_key)
  print("Natural Sort of Files:\t", input_list)

  ## Loop and Create a Table of Files in Folder
  print("\nFiles to Concatenate:")
  input_tables = {}
  input_log = pd.DataFrame(columns=['Region', 'FA Cost', 'NFA Rollup', 'Total Funding', 'Count', 'Rows', 'Columns', 'Sheet Name', 'File Name', 'File Path']) ## Clear/Create a dataframe
  for i, input in enumerate(input_list):

      ## Extract the region code from the filename
      region_code = input.split(".")[0] ## split the filename from the file type extension
      if "-" in region_code: ## for files named "R1-start.xls", etc.
        region_code = region_code.split("-")[0] ## keep only "R1", etc.

      ## Read the table
      input_path = os.path.join(input_folder, input)
      input_sheet_name = "STIP Project Listing" ## may need to update if format changes
      input_tables[region_code] = pd.read_excel(input_path, sheet_name=input_sheet_name)

      ## Extract the extra "Count:" row from each table
      count = int( input_tables[region_code][input_tables[region_code]['Phase Status'] == 'Count:']['Phase ID'].values[0] )
      input_tables[region_code] = input_tables[region_code][input_tables[region_code]['Phase Status'] != 'Count:'] ## Rows without the count row

      ## Convert 'FA Cost' and 'NFA Rollup' from millions to billions
      input_tables[region_code]['FA Cost'] = input_tables[region_code]['FA Cost'] / 1000
      input_tables[region_code]['NFA Rollup'] = input_tables[region_code]['NFA Rollup'] / 1000

      ## Add the region code to the table to distinguish in the compiled table
      input_tables[region_code]['Region'] = region_code
      input_tables[region_code].insert(0, 'Region', input_tables[region_code].pop('Region')) ## put this column in front of the other columns

      ## Add the date the STIP was last revised to distinguish in the compiled table
      input_tables[region_code]['Updated'] = update_date
      input_tables[region_code].insert(0, 'Updated', input_tables[region_code].pop('Updated')) ## put this column in front of the other columns

      ## Log the table
      try:
        input_log_row = {'Region': region_code,
                         'FA Cost': input_tables[region_code]['FA Cost'].sum(),
                         'NFA Rollup': input_tables[region_code]['NFA Rollup'].sum(),
                         'Total Funding': (input_tables[region_code]['FA Cost'].sum() + input_tables[region_code]['NFA Rollup'].sum()),
                         'Count': count,
                         'Rows': input_tables[region_code].shape[0],
                         'Columns': input_tables[region_code].shape[1],
                         'Sheet Name': input_sheet_name,
                         'File Name': input,
                         'File Path': input_path}
        input_log_row = pd.DataFrame(input_log_row, index=[i])
        input_log = pd.concat([input_log, input_log_row], axis=0, ignore_index=True)
      except:
        print(f"{i}\t{region_code}\t{input_tables[region_code].shape}\t'{input_sheet_name}' from {input} at '{input_path}'") ## Print line without table generation

      ## Add up the total number of rows in the compiled table
      if i == 0:
        total_rows = input_tables[region_code].shape[0]
        total_count = count
      else:
        total_rows = total_rows + input_tables[region_code].shape[0]
        total_count = total_count + count

  if input_log.shape[0] > 0: ## if the input log exists
    input_log.insert(4, 'FA Percent', input_log['FA Cost'] / input_log['Total Funding'] * 100) ## add FA percent column to the input log
    input_log.insert(5, 'NFA Percent', input_log['NFA Rollup'] / input_log['Total Funding'] * 100) ## add NFA percent column to the input log
    display(input_log) ## display it

  print(f"\n{len(input_tables)} total tables")
  print(f"{total_rows:,.0f} listed line items")
  print(f"{total_count:,.0f} count of line items")

  return input_tables

###### ▶ Inputs: STIP Table Folder

In [ ]:
"""
INPUTS: Update the following based on your data download:
"""
## 1. The new folder with your .xls files (and no other files).
input_folder = r"G:\Shared drives\Projects\5050_NYS Transportation\STIP\[2025-OCT-01] STIP Project List and Data Download (latest monthly list and download as of 10-01-2025)\2025-OCT-01 NYSDOT Region\xls"
## 2. Date the STIP was last updated (to use within the filename).
update_date = "2025-OCT-01"

In [131]:
## Use the function to load the tables as a list
input_tables = load_stip_tables(input_folder, update_date)

## Concatenate the STIP tables
df = pd.concat(input_tables, axis=0, ignore_index=True)
print("\nCompiled Table Dimensions:\t", df.shape)

## Check the Basic Metrics for the Compiled Stip Tables
df_totals = {
    'Federal Aid': df['FA Cost'].sum(),
    'Non-Federal Rollup': df['NFA Rollup'].sum(),
}
df_totals = pd.DataFrame.from_dict(df_totals, orient='index', columns=['Values'])
df_totals['Total'] = (df['FA Cost'].sum() + df['NFA Rollup'].sum())
df_totals['Percent'] = df_totals['Values'] / df_totals['Total'] * 100
display(df_totals)

display(df)

Initial Sort of Files:	 ['R1.xls', 'R10.xls', 'R11.xls', 'R2.xls', 'R3.xls', 'R4.xls', 'R5.xls', 'R6.xls', 'R7.xls', 'R8.xls', 'R9.xls', 'SW.xls']
Natural Sort of Files:	 ['R1.xls', 'R2.xls', 'R3.xls', 'R4.xls', 'R5.xls', 'R6.xls', 'R7.xls', 'R8.xls', 'R9.xls', 'R10.xls', 'R11.xls', 'SW.xls']

Files to Concatenate:


C:\Users\WillCao\AppData\Local\Temp\ipykernel_10656\3023911271.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  input_log = pd.concat([input_log, input_log_row], axis=0, ignore_index=True)


,Region,FA Cost,NFA Rollup,Total Funding,FA Percent,NFA Percent,Count,Rows,Columns,Sheet Name,File Name,File Path
0,R1,1.163121,0.788725,1.951846,59.590808,40.409192,1236,1236,16,STIP Project Listing,R1.xls,G:\Shared drives\Projects\5050_NYS Transportat...
1,R2,0.400105,0.167857,0.567962,70.445739,29.554261,742,742,16,STIP Project Listing,R2.xls,G:\Shared drives\Projects\5050_NYS Transportat...
2,R3,3.124192,0.993384,4.117576,75.874544,24.125456,996,996,16,STIP Project Listing,R3.xls,G:\Shared drives\Projects\5050_NYS Transportat...
3,R4,0.568620,0.321555,0.890175,63.877327,36.122673,726,726,16,STIP Project Listing,R4.xls,G:\Shared drives\Projects\5050_NYS Transportat...
4,R5,1.561282,1.493829,3.055111,51.103939,48.896061,1814,1814,16,STIP Project Listing,R5.xls,G:\Shared drives\Projects\5050_NYS Transportat...
5,R6,0.335440,0.182324,0.517764,64.786287,35.213713,597,597,16,STIP Project Listing,R6.xls,G:\Shared drives\Projects\5050_NYS Transportat...
6,R7,0.336212,0.084713,0.420925,79.874562,20.125438,612,612,16,STIP Project Listing,R7.xls,G:\Shared drives\Projects\5050_NYS Transportat...
7,R8,2.487980,8.564145,11.052125,22.511327,77.488673,1387,1387,16,STIP Project Listing,R8.xls,G:\Shared drives\Projects\5050_NYS Transportat...
8,R9,0.646891,0.286981,0.933872,69.269771,30.730229,960,960,16,STIP Project Listing,R9.xls,G:\Shared drives\Projects\5050_NYS Transportat...
9,R10,2.493620,4.568590,7.062210,35.309344,64.690656,721,721,16,STIP Project Listing,R10.xls,G:\Shared drives\Projects\5050_NYS Transportat...



12 total tables
10,656 listed line items
10,656 count of line items

Compiled Table Dimensions:	 (10656, 16)


,Values,Total,Percent
Federal Aid,35.377443,79.352456,44.582669
Non-Federal Rollup,43.975013,79.352456,55.417331


,Updated,Region,MPO,PIN,County,Air Quality,Resp Agency,Project Description,Short Description,Phase Type,FFY,FA Fund Type,FA Cost,NFA Rollup,Phase Status,Phase ID
0,2025-OCT-01,R1,AGFTC,111660,WARREN,EXEMPT,TOWN OF LAKE GEORGE,RECONSTRUCTION OF NYS 9N (LAKESHORE DRIVE) IN ...,"NYS 9N PEDESTRIAN AND BICYCLING IMPROVEMENTS, ...",PRELDES,2025.0,CRP MED URB,0.000200,0.000050,Obligated,108923
1,2025-OCT-01,R1,AGFTC,172270,WARREN,NaN,NYSDOT,"I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...","I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...",CONINSP,2026.0,NHPP,0.000492,0.000055,Programmed,112881
2,2025-OCT-01,R1,AGFTC,172270,WARREN,NaN,NYSDOT,"I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...","I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...",CONST,2026.0,NHPP,0.003281,0.000365,Programmed,112882
3,2025-OCT-01,R1,AGFTC,172270,WARREN,NaN,NYSDOT,"I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...","I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...",DETLDES,2025.0,NHPP,0.000164,0.000018,Programmed,47778
4,2025-OCT-01,R1,AGFTC,176101,WARREN,EXEMPT,CITY OF GLENS FALLS,"ACCESSIBILITY IMPROVEMENTS, 3300 FT TRAIL IMPR...",MAKE THE CONNECTION CITY OF GLENS FALLS - COLE...,CONINSP,2026.0,STBG FLEX,0.000129,0.000032,Programmed,112883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10651,2025-OCT-01,SW,SW,TSM025,SYSTEM-WIDE,EXEMPT,NYSDOT,"TO UPGRADE SIGNAL COMMUNICATIONS, USING A SW P...",STATEWIDE SIGNAL OPERATIONS AND DATA CONNECTIV...,OPER,2025.0,NHPP,0.009200,0.002300,Programmed,105095
10652,2025-OCT-01,SW,SW,TSM026,SYSTEM-WIDE,EXEMPT,NYSDOT,"TO UPGRADE SIGNAL COMMUNICATIONS, USING A SW P...",STATEWIDE SIGNAL OPERATIONS AND DATA CONNECTIV...,OPER,2026.0,CMAQ,0.013500,0.000000,Programmed,105098
10653,2025-OCT-01,SW,SW,TSM026,SYSTEM-WIDE,EXEMPT,NYSDOT,"TO UPGRADE SIGNAL COMMUNICATIONS, USING A SW P...",STATEWIDE SIGNAL OPERATIONS AND DATA CONNECTIV...,OPER,2026.0,NHPP,0.009200,0.002300,Programmed,105097
10654,2025-OCT-01,SW,SW,TSMV25,MULTI,EXEMPT,NYSDOT,TSMV25 - MIOVISION DETECTION PLUS UPGRADE FOR ...,TSMV25 - MIOVISION DETECTION PLUS UPGRADE,MISC,2025.0,HSIP,0.002000,0.000000,Obligated,112298


In [100]:
print( df['FA Fund Type'].unique() )

['CRP MED URB' 'NHPP' 'STBG FLEX' 'STBG-OFF' 'TAP RURAL' 'BFP OFF SYS' nan
 'FTA 5307' 'FTA 5339' 'FTA 5310' 'HSIP' 'STBG LG URB' 'TAP FLEX' 'HPP'
 'BFP MAIN' 'NHFP' 'CRP FLEX' 'TAP LG URBAN' 'CRP LG URB' 'CMAQ'
 'FTA 5309' 'OTHER FA' 'DISCR FA' 'FTA 5339 DIS' 'PROTECT' 'SAFE RT SCH'
 'TAP SM URBAN' 'FTA 5311' 'FTA 5339 ND' 'FTA NA' 'FTA MISC' 'REC TRAILS'
 'FTA 5337' 'FTA 5329' 'HSIP RAIL' 'STBG MED URB' 'STP ENHANCE' 'NEVI'
 'TAP MED URB' 'HRRR' 'FBP' 'FTA 5324' 'FTA 5309 NS' 'TIFIA']


# ➕ Add Data from Reference Tables

#### Single Column from Reference Table

##### Function: Map a single column

In [137]:
def map_from_ref_table(df, ref_keys_col, keys_col, ref_values_col, values_col, insert_index, ref_table_path):
  try:
    print(f"------------------------------\nAdding {values_col}:")
    df_ = df.copy() ## Back-up

    ## Load Table
    ref_table = pd.read_excel(ref_table_path)
    display(ref_table)

    ## Create Dictionary
    map_dict = dict(zip(ref_table[ref_keys_col], ref_table[ref_values_col])) ## create dictionary for mapping
    print(map_dict)

    ## Map Using Dictionary
    mapped_series = df[keys_col].map(map_dict) ## new pandas series with mapping

    ## Insert Column
    df_ = df_.drop(columns=[values_col], errors='ignore') ## drop old column
    df_.insert(insert_index, values_col, mapped_series) ## insert the column

    ## Add Counts to teh Reference Table
    ref_table = pd.merge(ref_table.set_index(ref_values_col), pd.DataFrame(df_[values_col].value_counts()), left_index=True, right_index=True, how='left')
    display(ref_table)

    if df_ is not None:
      return df_
    else:
      return df

  except Exception as e:
    print("Unable to map region name.\n", e)

###### ▶ Inputs: Region Reference Table

In [ ]:
"""
INPUTS: Update the following based on your data download:
"""
df = df
ref_keys_col = 'Region'
keys_col = 'Region'
ref_values_col = 'Region Name'
values_col = 'Region Name'
insert_index = 2
ref_table_path = r"G:\My Drive\25Q1_STIP\STIP_SCRIPTS\nys-stip-classification\input_reference_tables\STIP Regions List.xlsx"

In [135]:
## Execute the function
df = map_from_ref_table()
display(df)

------------------------------
Adding Region Name:


,index,Region,Region Code & Name,Region Name,Counties
0,0,SW,System-wide,System-wide,Multi
1,1,R1,Capital District (1),Capital District,"Albany, Essex, Greene, Rensselaer, Saratoga, S..."
2,2,R2,Mohawk Valley (2),Mohawk Valley,"Fulton, Hamilton, Herkimer, Madison, Montgomer..."
3,3,R3,Central New York (3),Central New York,"Cayuga, Cortland, Onondaga, Oswego, Seneca, To..."
4,4,R4,Genesee Valley (4),Genesee Valley,"Genesee, Livingston, Monroe, Ontario, Orleans,..."
5,5,R5,Western New York (5),Western New York,"Cattaraugus, Chautauqua, Erie, Niagara"
6,6,R6,Southern Tier Central (6),Southern Tier Central,"Allegany, Chemung, Schuyler, Steuben, Yates"
7,7,R7,North Country (7),North Country,"Clinton, Franklin, Jefferson, Lewis, St Lawrence"
8,8,R8,Hudson Valley (8),Hudson Valley,"Columbia, Dutchess, Orange, Putnam, Rockland, ..."
9,9,R9,Southern Tier (9),Southern Tier,"Broome, Chenango, Delaware, Otsego, Schoharie,..."


{'SW': 'System-wide', 'R1': 'Capital District', 'R2': 'Mohawk Valley', 'R3': 'Central New York', 'R4': 'Genesee Valley', 'R5': 'Western New York', 'R6': 'Southern Tier Central', 'R7': 'North Country', 'R8': 'Hudson Valley', 'R9': 'Southern Tier', 'R10': 'Long Island', 'R11': 'New York City'}


,index,Region,Region Code & Name,Counties,count
Region Name,,,,,
System-wide,0,SW,System-wide,Multi,193
Capital District,1,R1,Capital District (1),"Albany, Essex, Greene, Rensselaer, Saratoga, S...",1236
Mohawk Valley,2,R2,Mohawk Valley (2),"Fulton, Hamilton, Herkimer, Madison, Montgomer...",742
Central New York,3,R3,Central New York (3),"Cayuga, Cortland, Onondaga, Oswego, Seneca, To...",996
Genesee Valley,4,R4,Genesee Valley (4),"Genesee, Livingston, Monroe, Ontario, Orleans,...",726
Western New York,5,R5,Western New York (5),"Cattaraugus, Chautauqua, Erie, Niagara",1814
Southern Tier Central,6,R6,Southern Tier Central (6),"Allegany, Chemung, Schuyler, Steuben, Yates",597
North Country,7,R7,North Country (7),"Clinton, Franklin, Jefferson, Lewis, St Lawrence",612
Hudson Valley,8,R8,Hudson Valley (8),"Columbia, Dutchess, Orange, Putnam, Rockland, ...",1387


,Updated,Region,Region Name,MPO,PIN,County,Air Quality,Resp Agency,Project Description,Short Description,Phase Type,FFY,FA Fund Type,FA Cost,NFA Rollup,Phase Status,Phase ID
0,2025-OCT-01,R1,Capital District,AGFTC,111660,WARREN,EXEMPT,TOWN OF LAKE GEORGE,RECONSTRUCTION OF NYS 9N (LAKESHORE DRIVE) IN ...,"NYS 9N PEDESTRIAN AND BICYCLING IMPROVEMENTS, ...",PRELDES,2025.0,CRP MED URB,0.000200,0.000050,Obligated,108923
1,2025-OCT-01,R1,Capital District,AGFTC,172270,WARREN,NaN,NYSDOT,"I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...","I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...",CONINSP,2026.0,NHPP,0.000492,0.000055,Programmed,112881
2,2025-OCT-01,R1,Capital District,AGFTC,172270,WARREN,NaN,NYSDOT,"I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...","I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...",CONST,2026.0,NHPP,0.003281,0.000365,Programmed,112882
3,2025-OCT-01,R1,Capital District,AGFTC,172270,WARREN,NaN,NYSDOT,"I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...","I87 RESURFACING, EXIT 24-26, TOWN OF BOLTON, W...",DETLDES,2025.0,NHPP,0.000164,0.000018,Programmed,47778
4,2025-OCT-01,R1,Capital District,AGFTC,176101,WARREN,EXEMPT,CITY OF GLENS FALLS,"ACCESSIBILITY IMPROVEMENTS, 3300 FT TRAIL IMPR...",MAKE THE CONNECTION CITY OF GLENS FALLS - COLE...,CONINSP,2026.0,STBG FLEX,0.000129,0.000032,Programmed,112883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10651,2025-OCT-01,SW,System-wide,SW,TSM025,SYSTEM-WIDE,EXEMPT,NYSDOT,"TO UPGRADE SIGNAL COMMUNICATIONS, USING A SW P...",STATEWIDE SIGNAL OPERATIONS AND DATA CONNECTIV...,OPER,2025.0,NHPP,0.009200,0.002300,Programmed,105095
10652,2025-OCT-01,SW,System-wide,SW,TSM026,SYSTEM-WIDE,EXEMPT,NYSDOT,"TO UPGRADE SIGNAL COMMUNICATIONS, USING A SW P...",STATEWIDE SIGNAL OPERATIONS AND DATA CONNECTIV...,OPER,2026.0,CMAQ,0.013500,0.000000,Programmed,105098
10653,2025-OCT-01,SW,System-wide,SW,TSM026,SYSTEM-WIDE,EXEMPT,NYSDOT,"TO UPGRADE SIGNAL COMMUNICATIONS, USING A SW P...",STATEWIDE SIGNAL OPERATIONS AND DATA CONNECTIV...,OPER,2026.0,NHPP,0.009200,0.002300,Programmed,105097
10654,2025-OCT-01,SW,System-wide,SW,TSMV25,MULTI,EXEMPT,NYSDOT,TSMV25 - MIOVISION DETECTION PLUS UPGRADE FOR ...,TSMV25 - MIOVISION DETECTION PLUS UPGRADE,MISC,2025.0,HSIP,0.002000,0.000000,Obligated,112298


#### Multiple Columns from Reference Table

##### Function: Merge multiple columns

###### ▶ Inputs: Air Quality Reference Table

###### ▶ Inputs: Funding Source Reference Table

# 💬 Generate LLM Prompts

# ➗ Export Samples